# Bike-Share Demand Forecasting 1: Data Preparation

In this timeseries forecasting example we investigate demand in the [Capital Bikeshare scheme in 2011-12](https://www.capitalbikeshare.com/system-data), with relation to weather data.

This notebook downloads the [original weather-annotated dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) from the UC Irvine website, and performs some basic preparations before uploading to S3.

Later notebooks in this series fit models to the uploaded prepared data, and compare their accuracy.

### 1. Dataset 소개

 - 멤버쉽, 렌탈 및 자전거 반납 등의 프로세스가 자동으로 이루어지는 자전거 렌탈과 관련된 자전거 공유 시스템의 데이터입니다.
 - 데이터셋은 hour.csv와 day.csv를 가지고 있으며, 아래 구성 필드로 이루어져 있습니다. [단, day.csv는 hr(시간) 필드 제외]
    - instant: 기록된 index
    - dteday : 대여 날짜
    - season : 계절 데이터 (1:winter, 2:spring, 3:summer, 4:fall)
    - yr : 연도 (0: 2011, 1:2012)
    - mnth : 월 데이터 ( 1 to 12)
    - hr : 시각 (0 to 23)
    - holiday : 휴일 여부
    - weekday : 평일
    - workingday : 주중이면 1, 주말 또는 휴일이면 0
    - weathersit :
        - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
        - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
        - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
        - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
    - temp : Normalized 온도 (Celsius), 계산식 : (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
    - atemp: Normalized 체감 온도 (Celsius), 계산식 : (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
    - hum: Normalized 습도 (The values are divided to 100 (max))
    - windspeed: Normalized 풍속 (The values are divided to 67 (max))
    - casual: 미동록 사용자의 대여 횟수 
    - registered: 등록 사용자의 대여 횟수
    - cnt: 전체 자전가 대여 횟수 

## Dependencies and configuration<a class="anchor" id="setup"/>

As usual we start by loading libraries, defining configuration, and connecting to AWS SDKs

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install missingno

In [ ]:
%load_ext autoreload
%autoreload 1

# External Dependencies:
import boto3
import matplotlib.pyplot as plt
import pandas as pd
import missingno as msno
import seaborn as sn
import numpy as np

# Local Dependencies:
%aimport util

<p><img src='./BlogImages/add_img/dataset_com.png' width=700 height=600></p>



### CloudFormation으로 만들어진 S3 bucket 이름을 아래 넣습니다.
<p><img src='./BlogImages/add_img/s3bucket.png' width=700 height=500></p>

In [ ]:
bucket = 'forecast-demolab-XXX'# TODO: Choose a bucket you've created, and SageMaker has full access to
%store bucket

# Data files to be stored in S3:
data_prefix = "data/"
%store data_prefix
target_train_filename = "target_train.csv"
%store target_train_filename
target_test_filename = "target_test.csv"
%store target_test_filename
related_filename = "related.csv"
%store related_filename

Now we connect to our AWS SDKs, and initialise our access role (which may wait a little while to ensure any newly created permissions propagate):

In [ ]:
session = boto3.Session()
region = session.region_name
s3 = session.client(service_name="s3")

## Step 1: Fetch the source data<a class="anchor" id="fetch"/>

Since this data set is comparatively small, we can process it here in the notebook instance without requiring large disk allocations.

In [ ]:
!wget -O data.zip http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
!rm -rf ./data/raw
!mkdir -p ./data/raw
!unzip data.zip -d ./data/raw

The [UCI dataset documentation](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) tells us what to expect in terms of column headers, ranges and types, which we can load here:

In [ ]:
raw_df = pd.read_csv(
    "./data/raw/hour.csv",
    index_col="instant",
    dtype={
        "dteday": str,
        "season": int,
        "yr": int,
        "mnth": int,
        "hr": int,
        "holiday": bool,
        "weekday": int,
        "workingday": bool,
        "weathersit": int,
        "temp": float,
        "atemp": float,
        "hum": float,
        "windspeed": float,
        "casual": int,
        "registered": int,
        "cnt": int
    }
).sort_values(["dteday", "hr"])
raw_df.head()

...but we also have a quick check of the ranges and variation to check our expectations:

In [ ]:
raw_df.describe()

## Step 2: Source Data Analysis<a class="anchor" id="fetch"/>


First we will plot the overall demand across the lifetime of the full data set:

In [ ]:
hourly_df = raw_df.copy()

# Combine day and hour into datetime column, and drop superfluous date features:
hourly_df["dteday"] = pd.to_datetime(raw_df["dteday"].map(str) + " " + raw_df["hr"].map(str) + ":00")
hourly_df = hourly_df.rename(columns={ "dteday": "timestamp" }).drop(columns=["yr", "mnth", "hr"])

daterange = pd.date_range(
    start=list(hourly_df["timestamp"])[0],
    end=list(hourly_df["timestamp"])[-1],
    freq='H'
)
tmp = pd.DataFrame({ "timestamp": daterange })
tmp["dteday"] = tmp["timestamp"].dt.strftime("%Y-%m-%d")

fill_df = tmp.merge(
    hourly_df,
    how="left",
    on="timestamp"
).drop(columns=["dteday"])


In [ ]:
fig = plt.figure(figsize=(15, 8))
ax = plt.gca()
fill_df.plot(x="timestamp", y="registered", ax=ax, label="Registered Customers")
fill_df.plot(x="timestamp", y="casual", ax=ax, label="Casual Customers")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Trips")
ax.set_title("Comparison of Registered and Casual Demand - Whole Data Set")
plt.show()

In general we see from the above that:

1. 등록된 고객들이 대부분의 demand를 차지합니다.
2. 비록 여름/겨울 계절성을 데이터 내에서 볼 수 있지만, 서비스의 전체 증가 및 인기도는 2년 사이 전반적으로 성장하는 것을 볼 수 있습니다.
3. 전반적인 데이터에서 볼 수 있는 것 보다 단기간 내 강한 주기성 (spikiness)가 있습니다.

** 2번은 forecast의 정확도 관점에서 어려운 상황이긴 합니다. **
- 단지 2년의 데이터만을 가지고 있으며 연간 seasonaility가 중요하지만, 일반적으로는 중요 패턴에 대해서는 5배 더 긴 주기를 가는 과거 이력이 필요합니다.
- 서비스의 전반적인 수요는 non-stationary 한 것으로 판단되며, 이는 많은 forecasting 방법에서 정확도를 떨어뜨릴 수 있습니다.[here](https://cs.nyu.edu/~mohri/talks/NIPSTutorial2016.pdf)

- 아래 그래프에서는 수요에 대해 요일과 시간의 영향이 명확히 볼 수 있으며, 적어도 여름인 8월에 casual 라이더의 경우 주 보다 주말의 여행 비율이 높다는 것을 알 수 있습니다.
- 짧은 timescales에서는 데이터는 더욱 stationary 한 것을 볼 수 있고, 우리는 더욱 robust한 forecast 결과를 얻을 수 있습니다.

In [ ]:
period_df = fill_df[(fill_df["timestamp"] >= "2012-08-01") & (fill_df["timestamp"] < "2012-09-01")]
fig = plt.figure(figsize=(15, 8))
ax = plt.gca()
period_df.plot(x="timestamp", y="registered", ax=ax, label="Registered Customers")
period_df.plot(x="timestamp", y="casual", ax=ax, label="Casual Customers")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Trips")
ax.set_title("Comparison of Registered and Casual Demand - August 2012")
plt.show()

### 1. Data Trends and Outliers Analysis

In [ ]:
fig, axes = plt.subplots(nrows=11,ncols=2)
fig.set_size_inches(20, 60)
sn.boxplot(data=raw_df,y="casual",orient="v",ax=axes[0][0])
sn.boxplot(data=raw_df,y="registered",orient="v",ax=axes[0][1])
sn.boxplot(data=raw_df,y="casual",x="season",orient="v",ax=axes[1][0])
sn.boxplot(data=raw_df,y="registered",x="season",orient="v",ax=axes[1][1])
sn.boxplot(data=raw_df,y="casual",x="mnth",orient="v",ax=axes[2][0])
sn.boxplot(data=raw_df,y="registered",x="mnth",orient="v",ax=axes[2][1])
sn.boxplot(data=raw_df,y="casual",x="hr",orient="v",ax=axes[3][0])
sn.boxplot(data=raw_df,y="registered",x="hr",orient="v",ax=axes[3][1])
sn.boxplot(data=raw_df,y="casual",x="weekday",orient="v",ax=axes[4][0])
sn.boxplot(data=raw_df,y="registered",x="weekday",orient="v",ax=axes[4][1])
sn.boxplot(data=raw_df,y="casual",x="workingday",orient="v",ax=axes[5][0])
sn.boxplot(data=raw_df,y="registered",x="workingday",orient="v",ax=axes[5][1])

sn.boxplot(data=raw_df,y="casual",x="weathersit",orient="v",ax=axes[6][0])
sn.boxplot(data=raw_df,y="registered",x="weathersit",orient="v",ax=axes[6][1])
sn.boxplot(data=raw_df,y="casual",x="temp",orient="v",ax=axes[7][0])
sn.boxplot(data=raw_df,y="registered",x="temp",orient="v",ax=axes[7][1])
sn.boxplot(data=raw_df,y="casual",x="atemp",orient="v",ax=axes[8][0])
sn.boxplot(data=raw_df,y="registered",x="atemp",orient="v",ax=axes[8][1])
sn.boxplot(data=raw_df,y="casual",x="hum",orient="v",ax=axes[9][0])
sn.boxplot(data=raw_df,y="registered",x="hum",orient="v",ax=axes[9][1])
sn.boxplot(data=raw_df,y="casual",x="windspeed",orient="v",ax=axes[10][0])
sn.boxplot(data=raw_df,y="registered",x="windspeed",orient="v",ax=axes[10][1])

axes[0][0].set(ylabel='Casual',title="Box Plot On Casual")
axes[0][1].set(ylabel='Registered',title="Box Plot On Registered")
axes[1][0].set(xlabel='Season', ylabel='Casual',title="Box Plot On Casual Across Season")
axes[1][1].set(xlabel='Season', ylabel='Registered',title="Box Plot On Registered Across Season")
axes[2][0].set(xlabel='Months', ylabel='Casual',title="Box Plot On Casual Across Months")
axes[2][1].set(xlabel='Months', ylabel='Registered',title="Box Plot On Registered Across Months")
axes[3][0].set(xlabel='Hours', ylabel='Casual',title="Box Plot On Casual Across Hour Of The Day")
axes[3][1].set(xlabel='Hours', ylabel='Registered',title="Box Plot On Registered Across Hour Of The Day")
axes[4][0].set(xlabel='Weekday', ylabel='Casual',title="Box Plot On Casual Across Weekday")
axes[4][1].set(xlabel='Weekday', ylabel='Registered',title="Box Plot On Registered Across Weekday")
axes[5][0].set(xlabel='Working Day', ylabel='Casual',title="Box Plot On Casual Across Working Day")
axes[5][1].set(xlabel='Working Day', ylabel='Registered',title="Box Plot On Registered Across Working Day")

axes[6][0].set(xlabel='Weathersit', ylabel='Casual',title="Box Plot On Casual Across Weathersit")
axes[6][1].set(xlabel='Weathersit', ylabel='Registered',title="Box Plot On Registered Across Weathersit")
axes[7][0].set(xlabel='Temp', ylabel='Casual',title="Box Plot On Casual Across Temp")
axes[7][1].set(xlabel='Temp', ylabel='Registered',title="Box Plot On Registered Across Temp")
axes[8][0].set(xlabel='aTemp', ylabel='Casual',title="Box Plot On Casual Across aTemp")
axes[8][1].set(xlabel='aTemp', ylabel='Registered',title="Box Plot On Registered Across aTemp")
axes[9][0].set(xlabel='Hum', ylabel='Casual',title="Box Plot On Casual Across Hum")
axes[9][1].set(xlabel='Hum', ylabel='Registered',title="Box Plot On Registered Across Hum")
axes[10][0].set(xlabel='Windspeed', ylabel='Casual',title="Box Plot On Casual Across Windspeed")
axes[10][1].set(xlabel='Windspeed', ylabel='Registered',title="Box Plot On Registered Across Windspeed")

### 2. Missing Values Analysis

In [ ]:
msno.matrix(fill_df,figsize=(20,5))

### 3. Correlation Analysis

각 feature 대비 종속 변수가 어떻게 영향을 받는지 확인하기 위해 아래 상관 행렬을 만들어 봅니다.

   - 종속변수 ```cnt``` 대비하여 ```temp/atemp```는 양의 상관관계를 가지며, ```hum```은 음의 상관관계를 갖는 것을 확인할 수 있습니다.
   - ```windspeed```는 상관성이 낮은 것으로 판단됩니다.
   - ```atemp```는 ```temp```와 상호 높은 상관 관계를 가지므로 둘 중 하나의 데이터는 삭제하는 것이 좋습니다.  
     *multicollinearity 문제 : 독립변수 간의 상관관계가 발생할 경우 삭제하는 것이 좋음*
   - ```casual```과 ```registered```는 종속변수 ```cnt```를 분리한 값이므로 이후 모델링 시에는 삭제합니다.

In [ ]:
corrMatt = fill_df[["temp","atemp","casual","registered","hum","windspeed","cnt"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

## Step 3: Load data & interpolate gaps<a class="anchor" id="convert"/>

The raw dataset has:

* 날짜 / 시간 열 중복 제거 필요
* 대여 횟수가 없는 시간은 raw dataset에서 나타나지 않음

...which we'll deal with here, starting with the timestamp tidy-up:

In [ ]:
hourly_df = raw_df.copy()

# Combine day and hour into datetime column, and drop superfluous date features:
hourly_df["dteday"] = pd.to_datetime(raw_df["dteday"].map(str) + " " + raw_df["hr"].map(str) + ":00")
hourly_df = hourly_df.rename(columns={ "dteday": "timestamp" }).drop(columns=["yr", "mnth", "hr"])

아무도 자전거를 사용하지 않은 시간 동안 행은 누락되었지만 적용 범위 내 매일 최소 하나의 행은 존재합니다.
결측치는 아래와 같이 처리하였습니다.

 1. 라이더 수 (casual, registered, cnt) 는 0으로 설정 (cnt=0 은 행 대체가 적용된 경우 복구 시 사용)
 2. 일별 값 (season, holiday, weekday, workingday)은 해당 날짜의 원본 레코드의 평균값을 사용
 3. 날씨 데이터 (weathersit, temp, atemp, hum, windspeed): 시간 상 현재 레코드들과 가장 근접한 값들로 보간

날씨 데이터는 유일한 대체이며 시간별 보간으로 시간별 날씨 데이터의 결측 시간을 채우게 됩니다.

In [ ]:
# Compare the full range of hours covered by the data-set, to the number of records:
daterange = pd.date_range(
    start=list(hourly_df["timestamp"])[0],
    end=list(hourly_df["timestamp"])[-1],
    freq='H'
)

n_raw_records = len(hourly_df)
n_range_hours = len(daterange)
print(f"{n_raw_records} raw records vs {n_range_hours} hours in date range")
if (n_raw_records == n_range_hours):
    print("Data fully specified")
elif (n_raw_records < n_range_hours):
    # (We expect to see this)
    print("MISMATCH: Missing data will be interpolated")
elif (n_raw_records > n_range_hours):
    raise NotImplementedError("This script can't deal with duplicates yet!")

# Construct a fully range-covering table
# (including day-granularity fields taken from aggregating the source table)
tmp = pd.DataFrame({ "timestamp": daterange })
tmp["dteday"] = tmp["timestamp"].dt.strftime("%Y-%m-%d")
fill_df = tmp.merge(
    raw_df.groupby("dteday").agg("mean")[["season", "holiday", "weekday", "workingday"]],
    how="left",
    on="dteday"
).drop(columns=["dteday"])

# Join the whole-range table to our target, and fill in the day-granularity fields
assert (
    hourly_df.isna().sum().sum() == 0
), "These imputations assume no missing values in source data set records!"

imputed_df = fill_df.merge(hourly_df, how="left", on="timestamp", suffixes=("_day", ""))
imputed_df["season"].fillna(imputed_df["season_day"], inplace=True)
imputed_df["holiday"].fillna(imputed_df["holiday_day"], inplace=True)
imputed_df["weekday"].fillna(imputed_df["weekday_day"], inplace=True)
imputed_df["workingday"].fillna(imputed_df["workingday_day"], inplace=True)
imputed_df.drop(columns=["season_day", "holiday_day", "weekday_day", "workingday_day"], inplace=True)

# Fill all missing demand values with zero (which is why the records were missing in the first place)
imputed_df["casual"].fillna(0, inplace=True)
imputed_df["registered"].fillna(0, inplace=True)
imputed_df["cnt"].fillna(0, inplace=True)

# Interpolate over time for missing weather data fields:
imputed_df = imputed_df.set_index("timestamp")
imputed_df["weathersit"] = imputed_df["weathersit"].interpolate(method="time").round()
imputed_df["temp"].interpolate(method="time", inplace=True)
imputed_df["atemp"].interpolate(method="time", inplace=True)
imputed_df["hum"].interpolate(method="time", inplace=True)
imputed_df["windspeed"].interpolate(method="time", inplace=True)
imputed_df = imputed_df.reset_index()

assert not imputed_df.isna().any().any(), "Imputed DF should not have any remaining nulls!"
assert len(imputed_df) == n_range_hours, "Imputed DF should fully cover time range!"

print("Imputation complete")
imputed_df.head()

In [ ]:
# Feel free to extend feature engineering here, just leaving timestamp and *_demand columns alone:
target_suffix = "_demand"
full_df = imputed_df.rename(columns={
    "casual": f"casual{target_suffix}",
    "registered": f"registered{target_suffix}",
    "cnt": f"total{target_suffix}"
})

full_df.head()

## Step 4: Train/test split for eodel evaluation<a class="anchor" id="split"/>

Since a lot of feature engineering has been done for us already, the main outstanding question is how we'll evaluate the accuracy of our models.

### How to evaluate forecast models

forecasting 알고리즘을 평가하는 것과 일반적인 ML 방법과의 중요한 차이는 **casuality (인과관계)** 입니다. 과거 데이터 기반으로 우리의 모델을 fitting 해야 합니다. 그리고, 알수 없는 "미래" 데이터를 평가합니다. 모델의 fitting 프로세스에서는 미래의 데이터가 보이지 않도록 합니다.

**[Backtesting](https://en.wikipedia.org/wiki/Backtesting)** (or "hindcasting")는 이런 인관관계를 평가하는 프로세스입니다. 과거 데이터에서 시점 기준으로 하나 이상의 포인트를 선택하는 방법으로 cut-off 시나리오에서 사용합니다. 포인트 이전 데이터는 학습 데이터로 사용하고, 이후 데이터는 평가용으로 사용합니다.  

<img src="BlogImages/backtest.png"/>

다중의 timeseries를 갖는 상황에서, Target Timeseries와 Related Timeseries의 차이점을 아래 그림으로 알 수 있습니다.

* **target timeseries** (the things we want the model to forecast), and
* **[related timeseries](https://docs.aws.amazon.com/forecast/latest/dg/related-time-series-datasets.html)** (which we know ahead of time for our forecast window)

Therefore our train/test split will apply to the target timeseries only.

<img src="BlogImages/rts_viz.png">


### Choosing the Setup for our Bike Forecasting example

소스 데이터는 **hourly granularity**를 가지고 있으며, 이는 예측 가능한 단위의 가장 하한 값입니다.
이 granularity를 사용하거나 또는 aggregate up할지를 선택하는 것은 적절한 **forecast horizon**가 무엇인지를 가이드할 것입니다.

* 모델 학습은 주어진 forecast horizon에 대해 전체 정확도를 최적화할 것입니다. 따라서, 대부분의 유용한 horizon은 **비즈니스 문제**에 따라 선정해야 합니다. 예를 들어, 최선을 다해 1년을 예측하는 모델을 학습했다고 하지만, 실제 1개월 계획을 세우기 위해 사용하는 경우에는 성능이 떨어질 수 있습니다.
* DeepAR과 같은 RNN (Recurrent Neural Models)과도 비교할 것이기 때문에, granularity가 적어도 forecast window의 특정 비율일 때 이 아키텍처들의 일부에서 나쁘게 작동한다는 점도 고려할 필요가 있습니다. 어떤 패턴의 주기를 학습 할 수 있는지에 대한 제약 사항은 샘플 주기에 비례합니다. 특히, Amazon Forecast는 최대 500개의 샘플 forecast horizon까지 가능합니다. [limit](https://docs.aws.amazon.com/forecast/latest/dg/limits.html)
* 알고리즘은 context window 내 데이터가 거의 **stationary**하고 가장 긴 fluctuations의 여러 사이클을 잡아낼 때 최상으로 수행될 것입니다. 분기 또는 연간 forecasts는 사용 가능한 데이터셋에 대해 적합하지 않을 수 있습니다.

또한,

* (적어도 일부 국가에서) 괜찮은 날씨 예측이 가능한 기간과 수요에 적합하도록 bikes의 이동 및 배치를 위해 적합한 사전 planning 단계를 수행할 수 있는 기간 사이의 대략적인 평균을 고려하고,
* 시간별 샘플이 유지될 정도로 짧은 기간이며, 집계가 적절한 경우를 고려하여, 

이번에는 **14일 (2주) horizon**에 대한 predictors를 학습할 것입니다.

연말 기간이 business 적으로 중요한 시간임으로 가정하여, 2012-12-01 까지를 최종 테스트 cutoff로 offset합니다. final evaluation window는 단지 2년의 데이터를 제공하는 상황이기 때문에 모델에 대해 연간 seasonality의 노출을 최대화합니다.

<img src='BlogImages/add_img/dataset.png' width=900 height=700>

In [ ]:
cutoff_test_start = "2012-12-01"

full_headers = full_df.columns.to_list()
timestamp_header = "timestamp"
target_headers = list(filter(lambda s: s.endswith(target_suffix), full_headers))
target_nontotal_headers = list(filter(lambda s: s != "total_demand", target_headers))
related_headers = list(filter(lambda s: (s not in target_headers) and (s != timestamp_header), full_headers))

In [ ]:
# Unpivoted dataframe of target variables, still sorted by timestamp:
target_full_df = full_df[[timestamp_header] + target_nontotal_headers].melt(
    id_vars=[timestamp_header],
    value_vars=target_nontotal_headers,
    var_name="customer_type",
    value_name="demand"
).sort_values(by=[timestamp_header, "customer_type"]).reset_index(drop=True)

# Strip "_demand" from the target IDs:
target_full_df["customer_type"] = target_full_df["customer_type"].apply(lambda s: s[0:-len(target_suffix)])

target_full_df.head()

In [ ]:
target_train_df = target_full_df[target_full_df[timestamp_header] < cutoff_test_start]
target_test_df = target_full_df[target_full_df[timestamp_header] >= cutoff_test_start]

related_df = full_df[[timestamp_header] + related_headers]

print(f"{len(target_train_df)} target training points")
print(f"{len(target_test_df)} target test points")
print(f"{len(related_df)} related timeseries points")

related_df.head()

In [ ]:
print("Writing dataframes to file...")
target_train_df.to_csv(
    f"./data/{target_train_filename}",
    index=False
)
target_test_df.to_csv(
    f"./data/{target_test_filename}",
    index=False
)
related_df.to_csv(
    f"./data/{related_filename}",
    index=False
)

print("Uploading dataframes to S3...")
s3.upload_file(
    Filename=f"./data/{target_train_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{target_train_filename}"
)
s3.upload_file(
    Filename=f"./data/{target_test_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{target_test_filename}"
)
s3.upload_file(
    Filename=f"./data/{related_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{related_filename}"
)
print("Done")


Now that our data is uploaded to S3, we can train models and compare their performance! Move on to the next notebook to start fitting predictors.